### Targeted information retrieval

We have seen how to parse a webpage, which retrieves the information without distinction.

But, in general, the purpose of scraping is to automate the collection of targeted information on the web.


In [3]:
from bs4 import BeautifulSoup
import requests

Let's say I want to scrape the description of the latest movies released in theaters.

So we'll go to the Allociné website and try to find the tags that will give me links to the specific pages of these movies to get their summaries.

#### Recovery of the url of the pages of films newly released in the theaters

In [4]:
url = "http://www.allocine.fr/"
r = requests.get(url)
print(url, r.status_code)
soup = BeautifulSoup(r.content, "lxml")
soup

http://www.allocine.fr/ 200


<!DOCTYPE html>
<html lang="fr">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="index,follow,max-snippet:-1" name="robots"/>
<title>AlloCiné : Cinéma, Séries TV, BO de films et séries, Vidéos, DVD et VOD</title>
<meta content="" name="keywords"/>
<meta content="noarchive" name="Googlebot"/>
<meta content="global" name="distribution"/>
<meta content="AlloCine" name="author"/>
<meta content="France" name="country"/>
<meta content="48.87078;2.30447" name="geo.position"/>
<meta content="FR" name="geo.country"/>
<meta content="48.87078;2.30447" name="ICBM"/>
<meta content="#FECC00" name="theme-color"/>
<meta content="AlloCiné, le site de référence du cinéma et des séries tv ! Découvrez notre recherche d'horaires de films, le programme tv de vos séries préférées, l'actualité ciné et séries, les émissions

On your web browser (Chrome, Firefox,...), you can use the "inspect" function (right click -> inspect) and drag your mouse to the areas of the page that interest you. At the same time you will see the part of the HTML script highlighted that corresponds to the area that interests you.

That's how you find the tags that you are interested in.

I notice that the relative link of the web page specific to the new movie on the poster is stored in these tags:

```html
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=235582.html" title="Le Grand Bain">Le Grand Bain</a>
```


In [5]:
for p in soup.find_all("a"):
    print(p.text)

Un homme en colère
West Side Story
Lucifer
The Resident

Cinéma


Séries


Trailers


DVD


VOD


Kids

séances cinéma
news
dossiers
émissions AlloCiné
dernières bandes-annonces
Agenda des sorties
Films pour enfants à l'affiche
Meilleurs films
Meilleurs documentaires
Avant-premières
Box Office
Meilleurs films 2019
Tous les films pour enfants
Tous les films
Sorties de la semaine
Coupez !
Les Folies fermières
On sourit pour la photo
The Northman
Ima
Cœurs vaillants
The Duke
Utama : La Terre Oubliée
Nitram
L'école du bout du monde
Downton Abbey II : Une nouvelle ère
Doctor Strange in the Multiverse of Madness
Les Animaux Fantastiques : les Secrets de Dumbledore
Le Secret de la cité perdue
Tous les films encore à l'affiche
 Jurassic World: Le Monde d'après 
 
 Avatar : la voie de l'eau
 
 Firestarter
 
 Top Gun: Maverick
 
 Les Crimes du Futur
 
Tous les films à venir
Dernières news cinéma
De X-Men à Cannes, qui est Caleb Landry Jones, prix d'interprétation pour Nitram ?
TV Tips : la meill

This time, the site is more difficult to "extract". Let's use much more specific parameters to the search function `find_all`.

In [6]:
# In addition to the tag `a`, which is easily identifiable, we notice some additional
# information such as the value of the class variable of these identical tags.
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    print(elem)

<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=291483.html" title="Coupez !">Coupez !</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=288925.html" title="Les Folies fermières">Les Folies fermières</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=284745.html" title="On sourit pour la photo">On sourit pour la photo</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=278182.html" title="The Northman">The Northman</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=301011.html" title="Ima">Ima</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=285572.html" title="Cœurs vaillants">Cœurs vaillants</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=278710.html" title="The Duke">The Duke</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=293005.html" title="Utama : La Terre Oubliée">Utama : La Terre Oubliée</a

#### Recovery of `href`

We have noticed the presence of `href` links to the pages that interest us. Let's go get them back:

In [7]:
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    print(elem.get("href"))

/film/fichefilm_gen_cfilm=291483.html
/film/fichefilm_gen_cfilm=288925.html
/film/fichefilm_gen_cfilm=284745.html
/film/fichefilm_gen_cfilm=278182.html
/film/fichefilm_gen_cfilm=301011.html
/film/fichefilm_gen_cfilm=285572.html
/film/fichefilm_gen_cfilm=278710.html
/film/fichefilm_gen_cfilm=293005.html
/film/fichefilm_gen_cfilm=288592.html
/film/fichefilm_gen_cfilm=279377.html
/film/fichefilm_gen_cfilm=279664.html
/film/fichefilm_gen_cfilm=251390.html
/film/fichefilm_gen_cfilm=228087.html
/film/fichefilm_gen_cfilm=287082.html
/series/ficheserie_gen_cserie=28872.html
/series/ficheserie_gen_cserie=31219.html
/series/ficheserie_gen_cserie=25546.html
/series/ficheserie_gen_cserie=31595.html
/series/ficheserie_gen_cserie=29175.html
/series/ficheserie_gen_cserie=25633.html
/series/ficheserie_gen_cserie=25501.html


Can you retrieve the titles for me by searching for `"title"` in the items of the previous list?

In [8]:
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    print(elem.get("title"))

Coupez !
Les Folies fermières
On sourit pour la photo
The Northman
Ima
Cœurs vaillants
The Duke
Utama : La Terre Oubliée
Nitram
L'école du bout du monde
Downton Abbey II : Une nouvelle ère
Doctor Strange in the Multiverse of Madness
Les Animaux Fantastiques : les Secrets de Dumbledore
Le Secret de la cité perdue
Les 7 vies de Léa
En un battement
Moon Knight
Le Flambeau, les aventuriers de Chupacabra
Visions
Game Of Thrones: House of the Dragon
Star Wars: Obi-Wan Kenobi


#### Get summary

Let's start by building the URL that we will use to retrieve the summaries.

Start by putting the `href` values in a list of links.


In [9]:
links = []
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    links.append(elem.get("href"))
links

['/film/fichefilm_gen_cfilm=291483.html',
 '/film/fichefilm_gen_cfilm=288925.html',
 '/film/fichefilm_gen_cfilm=284745.html',
 '/film/fichefilm_gen_cfilm=278182.html',
 '/film/fichefilm_gen_cfilm=301011.html',
 '/film/fichefilm_gen_cfilm=285572.html',
 '/film/fichefilm_gen_cfilm=278710.html',
 '/film/fichefilm_gen_cfilm=293005.html',
 '/film/fichefilm_gen_cfilm=288592.html',
 '/film/fichefilm_gen_cfilm=279377.html',
 '/film/fichefilm_gen_cfilm=279664.html',
 '/film/fichefilm_gen_cfilm=251390.html',
 '/film/fichefilm_gen_cfilm=228087.html',
 '/film/fichefilm_gen_cfilm=287082.html',
 '/series/ficheserie_gen_cserie=28872.html',
 '/series/ficheserie_gen_cserie=31219.html',
 '/series/ficheserie_gen_cserie=25546.html',
 '/series/ficheserie_gen_cserie=31595.html',
 '/series/ficheserie_gen_cserie=29175.html',
 '/series/ficheserie_gen_cserie=25633.html',
 '/series/ficheserie_gen_cserie=25501.html']

The absolute URL of the movie pages we are looking for are built in this form: http://www.allocine.fr/film/fichefilm_gen_cfilm=243835.html

It is therefore necessary to take the previous list and build the absolute URLs for our search
It's up to you to play.

NB: Do not take the links for the shows(series). We only want movies.

In [10]:
movie_links = ["http://www.allocine.fr" + elem for elem in links if "film" in elem]
movie_links

['http://www.allocine.fr/film/fichefilm_gen_cfilm=291483.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=288925.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=284745.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=278182.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=301011.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=285572.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=278710.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=293005.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=288592.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=279377.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=279664.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=251390.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=228087.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=287082.html']

Finally, on each page, the title and synopsis must be retrieved. Let's try for a movie, the first of the list.

In [11]:
url = movie_links[0]
r = requests.get(url)
print(url, r.status_code)
soup = BeautifulSoup(r.content, "lxml")
soup

http://www.allocine.fr/film/fichefilm_gen_cfilm=291483.html 200


<!DOCTYPE html>
<html lang="fr">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="index,follow,max-snippet:-1,max-image-preview:standard" name="robots"/>
<title>Coupez ! - film 2022 - AlloCiné</title>
<meta content="" name="keywords"/>
<meta content="noarchive" name="Googlebot"/>
<meta content="global" name="distribution"/>
<meta content="AlloCine" name="author"/>
<meta content="France" name="country"/>
<meta content="48.87078;2.30447" name="geo.position"/>
<meta content="FR" name="geo.country"/>
<meta content="48.87078;2.30447" name="ICBM"/>
<meta content="#FECC00" name="theme-color"/>
<meta content="Coupez ! est un film réalisé par Michel Hazanavicius avec Matilda Lutz, Bérénice Bejo. Synopsis : Le film est présenté hors-compétition au Festival de Cannes 2022 et en fait l'ouverture.Un tournage de f

For title 
```html
<div class="titlebar-title titlebar-title-lg">Le Grand Bain</div>
```
For the synopsis

```html
<div class="content-txt " itemprop="description"

 
              C’est dans les couloirs de leur piscine municipale que Bertrand, Marcus, Simon, Laurent, Thierry et les autres s’entraînent sous l’autorité toute relative de Delphine, ancienne gloire des bassins. Ensemble, ils se sentent libres et utiles. Ils vont mettre toute leur énergie dans une discipline jusque-là propriété de la gent féminine : la natation synchronisée. Alors, oui c’est une idée plutôt bizarre, mais ce défi leur permettra de trouver un sens à leur vie...
    
      </div>
```

In [12]:
for elem in soup.find_all("div", attrs={"class": "titlebar-title titlebar-title-lg"}):
    # Just like that
    print(elem.text)
for elem in soup.find_all("div", attrs={"class": "content-txt"}):
    # Just like that
    print(elem.text)



Coupez !

Le film est présenté hors-compétition au Festival de Cannes 2022 et en fait l'ouverture.Un tournage de film de zombies dans un bâtiment désaffecté. Entre techniciens blasés et acteurs pas vraiment concernés, seul le réalisateur semble investi de l’énergie nécessaire pour donner vie à un énième film d'horreur à petit budget. L’irruption d’authentiques morts-vivants va perturber le tournage…



1) Automate this script for the entire list of movies.

2) Put the information in three lists (`film_links`, `title`, `synopsis`).

3) Create a `DataFrame` object from the `pandas` library that you will have to install. This dataframe will have to include these three informations in three columns.

4) Save this dataframe in a CSV file.

And here's your first real scrap, you're real hackers now.

In [16]:
import time
import random
from random import randint

title = []
synopsis = []

movie_links = ["http://www.allocine.fr" + elem for elem in links if "film" in elem]
#print(movie_links)

count = 0
for movie_url in movie_links:
    r = requests.get(movie_url)
    #print(movie_url, r.status_code)
    soup = BeautifulSoup(r.content, "lxml")

    for elem in soup.find_all("div", attrs={"class": "titlebar-title titlebar-title-lg"}):
        title.append(elem.text)
        print(title)

    for elem in soup.findAll("div", attrs={"class": "content-txt"}):
        if count == 0:
            print(count)
            print(elem.text)
            synopsis.append(elem.text)
            break

    time.sleep(random.uniform(1.0, 2.0))

# Check the length of the lists before creating the dataframe
len(title), len(synopsis), len(movie_links)
    
 




# Here are the things you will have to do for all links:
# - Slow down the frequency of requests to avoid being identified and therefore banned from the website.
# Use `time.sleep(random.uniform(1.0, 2.0))`
# - Get request object from URL
# - Extract the content into a variable using BeautifulSoup
# - Get title
# - Get synopsis

# Check the length of the lists before creating the dataframe
len(title), len(synopsis), len(movie_links)

http://www.allocine.fr/film/fichefilm_gen_cfilm=291483.html 200
['Coupez !']
0

Le film est présenté hors-compétition au Festival de Cannes 2022 et en fait l'ouverture.Un tournage de film de zombies dans un bâtiment désaffecté. Entre techniciens blasés et acteurs pas vraiment concernés, seul le réalisateur semble investi de l’énergie nécessaire pour donner vie à un énième film d'horreur à petit budget. L’irruption d’authentiques morts-vivants va perturber le tournage…

http://www.allocine.fr/film/fichefilm_gen_cfilm=288925.html 200
['Coupez !', 'Les Folies fermières']
0

D'après une fabuleuse histoire vraie.David, jeune paysan du Cantal, vient d’avoir une idée : pour sauver son exploitation de la faillite, il va monter un cabaret à la ferme. Le spectacle sera sur scène et dans l’assiette, avec les bons produits du coin. Il en est sûr, ça ne peut que marcher ! Ses proches, sa mère et surtout son grand-père, sont plus sceptiques.

http://www.allocine.fr/film/fichefilm_gen_cfilm=284745.ht

(14, 14, 14)

In [14]:
import pandas as pd

df = pd.DataFrame({"Title": title})
df["Synopsis"] = synopsis
df["Links"] = movie_links

In [15]:
df.to_csv("./assets/allo_cine.csv", index=False)